In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
dataset["classification_yes"].value_counts()

1    249
0    150
Name: classification_yes, dtype: int64

In [6]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes'  ]]
dep=dataset[['classification_yes' ]]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train=sc.fit_transform(X_train)
x_test=sc.transform(X_test)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid = {'criterion':['gini', 'entropy'],'max_features':['sqrt','log2'],'splitter':['best','random']}
grid = GridSearchCV( DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [10]:
results = pd.DataFrame(grid.cv_results_)
print(results[['params', 'mean_test_score', 'std_test_score']])

                                              params  mean_test_score  \
0  {'criterion': 'gini', 'max_features': 'sqrt', ...         0.950779   
1  {'criterion': 'gini', 'max_features': 'sqrt', ...         0.977315   
2  {'criterion': 'gini', 'max_features': 'log2', ...         0.955059   
3  {'criterion': 'gini', 'max_features': 'log2', ...         0.955001   
4  {'criterion': 'entropy', 'max_features': 'sqrt...         0.947490   
5  {'criterion': 'entropy', 'max_features': 'sqrt...         0.951210   
6  {'criterion': 'entropy', 'max_features': 'log2...         0.928832   
7  {'criterion': 'entropy', 'max_features': 'log2...         0.977518   

   std_test_score  
0        0.023034  
1        0.014164  
2        0.008974  
3        0.022512  
4        0.030303  
5        0.028225  
6        0.027434  
7        0.007475  


In [11]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003126,0.006251,0.009841,0.008079,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.962963,0.962264,0.925146,0.922492,0.981031,0.950779,0.023034,6
1,0.009349,0.007633,0.003126,0.006252,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",1.000000,0.961755,0.981217,0.981031,0.962573,0.977315,0.014164,2
2,0.009377,0.007656,0.006250,0.007654,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.945100,0.962264,0.962573,0.943093,0.962264,0.955059,0.008974,3
3,0.009452,0.005827,0.001600,0.003200,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.981569,0.943041,0.925524,0.943651,0.981217,0.955001,0.022512,4
4,0.011973,0.009514,0.000000,0.000000,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.944707,0.905069,0.944023,0.943651,1.000000,0.947490,0.030303,7
5,0.006402,0.003201,0.004798,0.003917,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.963284,0.924528,0.925146,0.943093,1.000000,0.951210,0.028225,5
6,0.006402,0.003201,0.004800,0.003919,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.945100,0.905069,0.888515,0.961826,0.943651,0.928832,0.027434,8
7,0.019202,0.018659,0.003201,0.003920,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.981569,0.981014,0.981217,0.981217,0.962573,0.977518,0.007475,1


In [12]:
bestParams=grid.best_params_
bestEstimator=grid.best_estimator_
bestScore=grid.best_score_
print("Best parameter {} ".format(bestParams))
print("Best Estimator {} ".format(bestEstimator))
print("Best Score {} ".format(bestScore))

Best parameter {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'random'} 
Best Estimator DecisionTreeClassifier(criterion='entropy', max_features='log2',
                       splitter='random') 
Best Score 0.97751800322885 


In [13]:
gridPred=grid.predict(x_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,gridPred)
print(cm)

[[47  4]
 [ 6 76]]


In [15]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,gridPred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.92      0.90        51
           1       0.95      0.93      0.94        82

    accuracy                           0.92       133
   macro avg       0.92      0.92      0.92       133
weighted avg       0.93      0.92      0.93       133



In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9241989478718318

In [17]:
from sklearn.metrics import f1_score
fscore=f1_score(y_test,gridPred,average='weighted')
print(fscore)

0.9250708680533242


In [18]:
indep.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes'],
      dtype='object')

In [20]:
age=int(input("AGE:"))
bp=float(input("BP:"))
al=float(input("AL:"))
su=int(input("SU:"))
bgr=float(input("BGR:"))
bu=float(input("BU:"))
sc=int(input("SC:"))
sod=float(input("SOD"))
pot=float(input("POT:"))
hrmo=float(input("HRMO:"))
pcv=int(input("PCV:"))
wc=int(input("WC:"))
rc=float(input("RC:"))
sg_b=int(input("SG B:"))
sg_c=int(input("SG C:"))
sg_d=int(input("SG D:"))
sg_e=int(input("SG E:"))
rbc_normal=int(input("RBC NORMAL:"))
pc_normal=int(input("PC NORMAL:"))
pcc_present=int(input("PC PRESENT:"))
ba_present=int(input("BA PRESENT:"))
htn_yes=int(input("HTN YES:"))
dm_yes=int(input("DM YES:"))
cad_yes=int(input("CAD YES:"))
appet_yes=int(input("APPET YES:"))
pe_yes=int(input("PE YES"))
ane_yes=int(input("ANE YES:"))

AGE:21
BP:55.34
AL:2.3
SU:2
BGR:89.3
BU:34.3
SC:2
SOD120.4
POT:2.3
HRMO:7.4
PCV:23
WC:1200
RC:2.5
SG B:0
SG C:1
SG D:0
SG E:0
RBC NORMAL:1
PC NORMAL:0
PC PRESENT:1
BA PRESENT:1
HTN YES:1
DM YES:0
CAD YES:0
APPET YES:1
PE YES1
ANE YES:0


In [21]:
Prediction=grid.predict([[age,bp,al,su, bgr, bu, sc, sod, pot, hrmo, pcv,
       wc, rc, sg_b, sg_c, sg_d, sg_e, rbc_normal, pc_normal,
       pcc_present, ba_present, htn_yes, dm_yes, cad_yes,
       appet_yes, pe_yes, ane_yes]])

In [22]:
print(" Prediction Value : {} ".format(Prediction))

 Prediction Value : [1] 
